# Trustpilot - Pony Challenge

In [323]:
import requests
import json
from matplotlib import pyplot as plt
import numpy
from IPython.display import Image
from IPython.core.display import HTML 


#Global Maze Data:
maze_width = 15
maze_height = 15

In [324]:
data = {"maze-width": maze_width,'maze-height': maze_height,'maze-player-name':'Applejack','difficulty': 0}

In [325]:
headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}

## Create the maze with a valid Pony name as parameter

In [326]:
create_maze_response = requests.post('https://ponychallenge.trustpilot.com/pony-challenge/maze', json=data, headers=headers)

In [327]:
print(create_maze_response.text)

{"maze_id":"91eb7f74-46ec-4286-a446-f4dc9960c86e"}


In [328]:
json_response = create_maze_response.json()
print(json_response)


{'maze_id': '91eb7f74-46ec-4286-a446-f4dc9960c86e'}


In [329]:
maze_id = json_response['maze_id']

In [330]:
get_maze_response = requests.get('https://ponychallenge.trustpilot.com/pony-challenge/maze/{0}'.format(maze_id), headers=headers)

In [331]:
get_maze_response_json = get_maze_response.json()

## Checking walkable paths:

## Documentation directions:

### "The 'data' contains an array with width*height entries. Each entry has at most 2 walls, 'west' and 'north'. If you want to find all walkable directions from place X you need to use the array entries X, X+1 and X+width to construct all walls around the place X."

In [332]:
pony_position = get_maze_response_json['pony'][0]

In [333]:
domokun_position = get_maze_response_json['domokun'][0]

In [334]:
end_point = get_maze_response_json['end-point'][0]

In [335]:
def check_connected_nodes(entries, entry_index):
    
    east_entry = entry_index + 1
    south_entry = entry_index + maze_width
    north_entry = entry_index - maze_width 
    west_entry = entry_index - 1
    
    possible_connections = []
    
    if east_entry <= len(entries) - 1:
        possible_connections.append(east_entry)
    if west_entry >= 0:
        possible_connections.append(west_entry)
    if south_entry <= len(entries) - 1:
        possible_connections.append(south_entry)
    if north_entry >= 0:
        possible_connections.append(north_entry)

    if 'north' in entries[entry_index] and north_entry in possible_connections:
        possible_connections.remove(north_entry)
    if 'west' in entries[entry_index] and west_entry in possible_connections:
        possible_connections.remove(west_entry)

    # East Entry Check
    if east_entry in possible_connections:
        if 'west' in entries[east_entry]:
            # remove east as possible direction
            possible_connections.remove(east_entry)

    # South entry check
    if south_entry in possible_connections:
        if 'north' in entries[south_entry]:
            possible_connections.remove(south_entry)
    
    return possible_connections
    
    

In [336]:
entries = get_maze_response_json['data']

#Let's start to build a graph with this data. Based on the possible directions for eacht entry, we will add the graph nodes.
graph = {}

for index, entry in enumerate(entries):
    graph[index] = check_connected_nodes(entries, index)
    



In [337]:
print(pony_position)

65


In [338]:
print(domokun_position)

70


In [339]:
# Graph Modeling and path search recursive strategy as seen at: https://www.python.org/doc/essays/graphs/
def find_path(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return path
    if start not in graph:
        return None
    for node in graph[start]:
        if node not in path:
            newpath = find_path(graph, node, end, path)
            if newpath: return newpath
    return None

In [340]:
def convert_to_directions(path):
    directions = []
    for index, entry in enumerate(path):
        if(index + 1) > (len(path) -1):
            continue
        if path[index + 1] - path[index] == -1:
            directions.append('west')
        elif path[index + 1] - path[index] == +1:
            directions.append('east')
        elif path[index + 1] - path[index] == maze_width:
            directions.append('south')
        elif path[index + 1] - path[index] == -maze_width:
            directions.append('north')
    return directions

In [341]:
result = find_path(graph, pony_position, end_point)

In [342]:
print(result)

[65, 50, 35, 36, 51, 52, 37, 38, 39, 54, 69, 68, 67, 66, 81, 82, 83, 98, 99, 114, 129, 128, 143, 144, 145, 146, 161, 160, 159, 158, 173, 174, 175, 190, 189, 204, 219, 218]


In [343]:
directions = convert_to_directions(result)

In [344]:
print(directions)

['north', 'north', 'east', 'south', 'east', 'north', 'east', 'east', 'south', 'south', 'west', 'west', 'west', 'south', 'east', 'east', 'south', 'east', 'south', 'south', 'west', 'south', 'east', 'east', 'east', 'south', 'west', 'west', 'west', 'south', 'east', 'east', 'south', 'west', 'south', 'south', 'west']


In [345]:
## With the directions list, move the poney until reach the end of the maze.

In [346]:
for direction in directions:
    data = { 'direction' : direction}
    move_response = requests.post('https://ponychallenge.trustpilot.com/pony-challenge/maze/{0}'.format(maze_id), json=data, headers=headers)
    print(move_response.text)

{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}
{"state":"active","state-result":"Move accepted"}


In [347]:
Image(url= "https://ponychallenge.trustpilot.com/eW91X3NhdmVkX3RoZV9wb255.jpg")